In [84]:
class UnitVec():
    pass

def IntVec():
    def __init__(self, labelfun, amps):
        pass
class ListVec(list):
    ''' VecList stores vector in form [(vec, amp)]'''
    def bind(self,f):
        retvec = ListVec()
        for k,a1 in self:
             retvec += map(lambda k2, a2: (k2,a2 * a1) , f(k))
        return retvec
    def fmap(self,f):
        return ListVec([(f(k), a) for k,a in self])
    def lookup(self, key):
        for k,a in self:
            if k == key:
                return a
    def unit(key):
        return ListVec([(key,1.0)])
            
class HashVec(dict):
    pass
#Trick a: We can use Kron to replace scalar multiplication

class KronVec(tuple):
    def __new__(self, a , b):
        return tuple.__new__(self, (a, b))
    def fst(self):
        return self[0]
    def tensor(self): #This is assuming we have bare lists?
        return ListVec([ ((k1,k2), a1 * a2) for k1,a1 in self[0] for k2,a2 in self[1]])
    def _ab_c(self):
        return KronVec( self[0][0],  KronVec(self[0][1], self[1]))
    def a_bc_(self):
        return KronVec(KronVec(self[0],self[1][0]), self[1][1])
    def __iter__(self):
        return self.tensor().__iter__()
    def lmap(self, f):
        return KronVec(f(self[0]), self[1])
    def rmap(self, f):
        return KronVec(self[0], f(self[1]))
             
#KronVec = namedtuple('KronVec', ('a','b'))
    # A holder for (v1,v2) pairs
    
class KronList(list):
    pass

def qubits(n):
    nullvec = ListVec.unit(0)
    if n > 1:
        return KronVec( nullvec, qubits(n-1))
    else:
        return nullvec


def cnot(g1, g2):
    def cnot2(label):
        label[g2] = label[g2] != label[g1] #not equal is xor
        return label
    return cnot2

def unit(label):
    return [(label, 1.0)]
def liftM(f):
    return lambda label: unit(f(label))

def qcnot(g1, g2):
    return liftM(cnot(g1,g2))
def bind(v,f):
    return [f()]

def kron():
    pass
    
def tensor():
    pass# Take a kroned object and pushes the pairing inside
    
def assoc():
    pass#reassociates the untensed kron
    

In [1]:
from collections import namedtuple


In [9]:
ListVec([('a',2.0)]).lookup('a')

2.0

In [11]:
ListVec.unit('a').lookup('a')

1.0

In [36]:
KronVec(ListVec([('a',2.0), ('b',3.0),('e',3.0)]),  ListVec([('c',5.0), ('d',3.0)]) ).tensor()

[(('a', 'c'), 10.0),
 (('a', 'd'), 6.0),
 (('b', 'c'), 15.0),
 (('b', 'd'), 9.0),
 (('e', 'c'), 15.0),
 (('e', 'd'), 9.0)]

In [42]:
vec = KronVec(ListVec([('a',2.0), ('b',3.0),('e',3.0)]),  ListVec([('c',5.0), ('d',3.0)]) )
for k, a in vec:
    print(k)

('a', 'c')
('a', 'd')
('b', 'c')
('b', 'd')
('e', 'c')
('e', 'd')


In [65]:
qubits(4)

(((0, (0, 0)), 1.0))

In [69]:
ListVec.unit(0).lift(lambda x: x)

[(3, 1.0)]

In [76]:
def classical_cnot(c):
    a,b = c
    return (a , (a+b) % 2)

In [77]:
classical_cnot((1,1))

(1, 0)

In [88]:
qubits(2).lmap(lambda a: a.fmap(lambda z : z +1)).tensor().fmap(classical_cnot)
qubits(2).lmap(lambda a: a.fmap(lambda z : z +1)).tensor().fmap(classical_cnot).fmap(classical_cnot)

[((1, 0), 1.0)]

why not use a hash vect? Because of the immutable requirement? But we cna use object id?

Interesting... Factorization of classical expression can be very useful.
shannon expansion?
BDD?
You kind of want them sepearated into don't care blocks. forall?
the doesn't matter does allows for us to maintain kron form?

Yea. I kind of want to index bits